## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,89.82,55,40,4.27,scattered clouds
1,1,Mataura,NZ,-46.1927,168.8643,49.48,96,99,1.86,overcast clouds
2,2,Iqaluit,CA,63.7506,-68.5145,34.70,74,94,5.99,light rain
3,3,New Norfolk,AU,-42.7826,147.0587,46.71,82,100,1.99,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,73.99,66,0,13.35,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,89.82,55,40,4.27,scattered clouds
6,6,Thinadhoo,MV,0.5333,72.9333,84.27,71,88,18.03,overcast clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.54,85,83,7.00,broken clouds
19,19,Trat,TH,12.5000,102.5000,78.87,89,6,1.61,clear sky
21,21,Tecoanapa,MX,16.5167,-98.7500,88.00,59,89,5.79,overcast clouds
22,22,Pemangkat,ID,1.1667,108.9667,81.14,79,18,3.49,few clouds
23,23,Tabou,CI,4.4230,-7.3528,77.68,90,98,8.48,moderate rain
24,24,Rairangpur,IN,22.2676,86.1751,88.41,49,100,3.31,overcast clouds
29,29,Georgetown,MY,5.4112,100.3354,85.93,74,20,0.92,light rain
30,30,Ribeirao Bonito,BR,-22.0667,-48.1761,80.35,45,91,5.82,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cabo San Lucas,MX,89.82,scattered clouds,22.8909,-109.9124,
6,Thinadhoo,MV,84.27,overcast clouds,0.5333,72.9333,
9,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,
19,Trat,TH,78.87,clear sky,12.5000,102.5000,
21,Tecoanapa,MX,88.00,overcast clouds,16.5167,-98.7500,
22,Pemangkat,ID,81.14,few clouds,1.1667,108.9667,
23,Tabou,CI,77.68,moderate rain,4.4230,-7.3528,
24,Rairangpur,IN,88.41,overcast clouds,22.2676,86.1751,
29,Georgetown,MY,85.93,light rain,5.4112,100.3354,
30,Ribeirao Bonito,BR,80.35,overcast clouds,-22.0667,-48.1761,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
     # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cabo San Lucas,MX,89.82,scattered clouds,22.8909,-109.9124,Tesoro Los Cabos
9,Puerto Ayora,EC,75.54,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
19,Trat,TH,78.87,clear sky,12.5000,102.5000,สวนหนิง-อัศ
21,Tecoanapa,MX,88.00,overcast clouds,16.5167,-98.7500,Hotel Manglares
22,Pemangkat,ID,81.14,few clouds,1.1667,108.9667,Ramah Kost KITA
23,Tabou,CI,77.68,moderate rain,4.4230,-7.3528,hotêl doufoulougou
24,Rairangpur,IN,88.41,overcast clouds,22.2676,86.1751,Rairangpur Town PS
29,Georgetown,MY,85.93,light rain,5.4112,100.3354,Cititel Penang
30,Ribeirao Bonito,BR,80.35,overcast clouds,-22.0667,-48.1761,Hotel Pousada Veraneio
34,Faanui,PF,78.57,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [38]:
# 8a. Create the output File (CSV)
output_data_file= "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [50]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))